In [ ]:
import os
import time
import cv2
import numpy as np
import my_yolo

In [ ]:
def process_image(img):
    image = cv2.resize(img, (416, 416), interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [ ]:
def get_classes(file):
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [ ]:
def draw(image, boxes, scores, classes, all_classes):
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image,all_classes[cl],
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

In [ ]:
def detect_image(image, all_classes):
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = my_yolo.prediction(pimage, image.shape)
    end = time.time()
    print('Inference time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [ ]:
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

for (root, dirs, files) in os.walk('images/input'):
    if files:
        for f in files:
            print("File name: ",f)
            path = os.path.join(root, f)
            image = cv2.imread(path)
            image = detect_image(image, all_classes)
            cv2.imwrite('images/result/1' + f, image)


In [ ]:
cv2.imshow("Image",image)
cv2.waitKey(0)

In [ ]:
def detect_video(video, all_classes):
    video_path = os.path.join("videos", "test", video)
    vcap = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    dim = (int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    ext = cv2.VideoWriter_fourcc(*'mp4v')

    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "result", video), ext, 20, dim, True)

    while True:
        res, frame = vcap.read()

        if not res:
            break

        image = detect_image(frame, all_classes)
        cv2.imshow("detection", image)

        vout.write(image)

        if cv2.waitKey(100) == 27:
            cv2.destroyAllWindows()

    vout.release()
    vcap.release()


In [ ]:
#video = 'Indian_Traffic_Trim.mp4'
#detect_video(video,get_classes('data/coco_classes.txt'))